In [1]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, download_plotlyjs, plot, iplot

import seaborn as sns
sns.set()

In [2]:
import re
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['texto'].apply(lambda text: get_emojis(text))


def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords])
    return texto

data['pp'] = data['texto'].apply(lambda texto: pre_procesado(texto))


data

NameError: name 'data' is not defined

In [ ]:
from collections import Counter
from nltk.util import ngrams 

In [ ]:
palabras_pp = data.pp.str.cat(sep=" ").split()
bigramas = pd.DataFrame(Counter(ngrams(palabras_pp,2)).most_common())
bigramas.columns = ['texto', 'cnt']
bigramas.texto = bigramas.texto.apply(lambda val: " ".join(val))
bigramas = bigramas[bigramas['cnt']>25]

In [ ]:
trace1 = go.Bar(x = bigramas.texto.values,
                y = bigramas.cnt.values)

traces = [trace1]

layout = go.Layout(title='Bigramas',
                   xaxis=dict(title='Bigrama'),
                   yaxis=dict(title='Cuántas ocurrencias')
                  )

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [ ]:
trace1 = go.Bar(x = bigramas.texto.values,
                y = bigramas.cnt.values,
                marker=dict(color='rgb(224,180,158)', line=dict(color='rgb(90,55,6)',width=1.5)),
                hovertemplate =
                '<i>Bigrama</i>: %{x}'+
                '<br><b>Contador</b>: %{y}<br>',
                text = [f'Custom text {i+1}' for i in range(5)],
                name='Bigramas'
               )

traces = [trace1]

layout = go.Layout(title='Bigramas',
                   xaxis=dict(title='Bigrama'),
                   yaxis=dict(title='Cuántas ocurrencias'),
                  )

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [ ]:
emojis_ciudad = []

for ciudad in data.ciudad.unique():
    try:
        temp = data[data.ciudad == ciudad]
        palabras_pp = temp.emojis.apply(lambda val: " ".join(val)).str.cat(sep=" ").split()
        bigramas = pd.DataFrame(Counter(ngrams(palabras_pp,1)).most_common())
        bigramas.columns = ['texto', 'cnt']
        bigramas.texto = bigramas.texto.apply(lambda val: " ".join(val))
        bigramas = bigramas.head(10)
        bigramas['ciudad'] = ciudad
        emojis_ciudad.append(bigramas)
    except:
        Noneb

In [ ]:
listOfFrames = []
traces = []

for ec in emojis_ciudad:
    ciudad = ec.ciudad.unique()[0]
    trace1 = go.Bar(x = ec.texto.values,
                    y = ec.cnt.values,
                    textposition = "outside",
                    texttemplate = "%{x}<br>%{y}",
                    textfont = {'size': [14]*10},
                    marker=dict(color='orange', line=dict(color='rgb(224,180,158)',width=1.5)),
                    hovertemplate =
                    '<br><i>Bigrama</i>: %{x}'+
                    '<br><b>Contador</b>: %{y}<br>',
                    text = [f'Custom text {i+1}' for i in range(5)],
                    name=ciudad
                   )
    traces.append(trace1)
    
    layout = go.Layout(title=f'Emojis en {ciudad}',
                       xaxis = {"range":[-1,10],"autorange": False, "showline":False,"tickangle":-90, "visible":False},
                       yaxis = {"range":[-1,30],"autorange": False, "showline":False, "visible":False},
                       plot_bgcolor= '#FFFFFF')
    listOfFrames.append(go.Frame(data=[trace1], layout=layout))
    
frames=list(listOfFrames)




layout = go.Layout(title=f'Emojis in {ciudad}',
                   xaxis = {"range":[-1,10],"autorange": False, "showline":False,"tickangle":-90, "visible":False},
                   yaxis = {"range":[-1,30],"autorange": False, "showline":False, "visible":False},
                   plot_bgcolor= '#FFFFFF',
                   updatemenus=[dict(type="buttons",
                                     buttons=[dict(label="▶️",
                                                   method="animate",
                                                   args=[None, {"frame": {"duration": 800, "redraw": True},
                                                                "fromcurrent": True}]),
                                              {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "⏸️",
                "method": "animate"
            }
                                             ]
                                    )
                               ]
                  )

fig = go.Figure(data=[traces[0]], layout=layout, frames=frames)

fig.show()